<img style="float: left; margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="300" height="500" /> 
    
    
### <font color='navy'> Simulación de procesos financieros

**Nombres:**
Mateo Verea Dorantes,
Alejandra Rico

**Fecha:** 22 de noviembre de 2021

**Expediente** : 
    if709396
    
**Profesor:** Oscar David Jaramillo Zuluaga
    
**Link Github**:https://github.com/Mateoverea/Tarea9-10_MVerea_ARico
    
# Tarea 9

Implementar el método de esquemas del trapecio, para valuar la opción call y put asiática con precio inicial, $S_0 = 100$, precio de ejercicio $K = 100$, tasa libre de riesgo $r = 0.10$, volatilidad $\sigma = 0.20$ y $T = 1$ año. Cuyo precio es $\approx 7.04$. Realizar la simulación en base a la siguiente tabla:
![imagen.png](attachment:imagen.png)

Observe que en esta tabla se encuentran los intervalos de confianza de la aproximación obtenida y además el tiempo de simulación que tarda en encontrar la respuesta cada método. 
- Se debe entonces realizar una simulación para la misma cantidad de trayectorias y número de pasos y construir una Dataframe de pandas para reportar todos los resultados obtenidos.**(70 puntos)**
- Compare los resultados obtenidos con los resultados arrojados por la función `Riemann_approach`. Concluya. **(30 puntos)**

#### Código de solución `ESTUDIANTE 1: ALEJANDRA RICO`

In [1]:
# Importamos librerías a utilizar.
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import datetime
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
import time
%matplotlib inline

# Algunas opciones para Pandas.
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 9)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 78)
pd.set_option('precision', 3)

In [2]:
# Función para simular precios Black-Scholes.
def BSprices(mu,sigma,S0,NbTraj,NbStep):
    """
    Expresión de la solución de la ecuación de Black-Scholes
    St = S0*exp((r-sigma^2/2)*t+ sigma*DeltaW)
    
    Parámetros
    ---------
    mu    : Tasa libre de riesgo
    sigma : Desviación estándar de los rendimientos
    S0    : Precio inicial del activo subyacente
    NbTraj: Cantidad de trayectorias a simular
    NbStep: Número de días a simular
    """
    # Datos para la fórmula de St
    nu = mu-(sigma**2)/2
    DeltaT = 1/NbStep
    SqDeltaT = np.sqrt(DeltaT)
    DeltaW = SqDeltaT*np.random.randn(NbTraj,NbStep-1)
    
    # Se obtiene --> Ln St = Ln S0+ nu*DeltaT + sigma*DeltaW
    increments = nu*DeltaT + sigma*DeltaW
    concat = np.concatenate((np.log(S0)*np.ones([NbTraj,1]),increments),axis=1)
    
    # Se utiliza cumsum por que se quiere simular los precios iniciando desde S0
    LogSt = np.cumsum(concat,axis=1)
    # Se obtienen los precios simulados para los NbStep fijados
    St = np.exp(LogSt)
    # Vector con la cantidad de días simulados
    t = np.arange(0,NbStep)

    return St.T,t

$$\hat V_0^{(2)}= {e^{-rT} \over M} \sum_{j=1}^{M} \Bigg({h\over 2T} \sum_{i=0}^{N-1} S_{t_i}(2+rh+(W_{t_{i+1}}-W_{t_i})\sigma)-K \Bigg)_+,$$

In [3]:
# Función Método del Trapecio.
def Trapecio(K,r,S0,NbTraj,NbStep,sigma,T,Option_Type,Trust_level,flag=None):
    """
    K:           Strike price
    r:           Tasa libre de riesgo
    S0:          Precio inicial
    NbTraj:      Número trayectorias
    NbStep:      Cantidad de pasos a simular
    sigma:       Volatilidad
    T:           Tiempo de cierre del contrato en años
    Option_Type: Tipo de opción a valuar
    Trust_level: Confianza definida para el intervalo de precios
    """
    # Definimos tiempo de ejecución
    start = time.time()
    
    # Resolvemos la ecuación de black scholes para obtener los precios
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    # Almacenamos los precios en un dataframe
    prices = pd.DataFrame(St,index=t)
    # Definimos la longitud h
    h = T / NbStep
    # Definimos el dataframe de strikes
    strike = K
    
    if Option_Type in ['Call', 'call']:
        # Definimos el trapecio para el call
        Average_trapeze_c = (prices * (2 + (r*h) + ((np.random.randn(NbStep, NbTraj)) * sigma))).expanding().sum()
        
        # Calculamos el call de la opción según la formula obtenida para el trapecio
        call = pd.DataFrame({'Prima': np.exp(-r*T) \
                     *np.fmax((h/2*T) * Average_trapeze_c - strike, 0).mean(axis=1)}, index=t)
        # Intervalo de confianza
        sigma_est_call = call.sem().Prima
        mean_est_call = call.iloc[-1].Prima
        i1_call = st.norm.interval(Trust_level, loc=mean_est_call, scale=sigma_est_call)
        
        end_c = time.time()
        total_time_c = end_c - start

        return call.iloc[-1].Prima, i1_call[0], i1_call[1], i1_call[1] - i1_call[0], total_time_c
    
    else:
        # Definimos el trapecio para el put
        Average_trapeze_p = (prices * (2 + (r*h) + ((np.random.randn(NbStep, NbTraj)) * sigma))).expanding().sum()
        
        # Calculamos el put de la opción según la formula obtenida para el trapecio
        put = pd.DataFrame({'Prima': np.exp(-r*T) \
                     *np.fmax(strike - (h/2*T) * Average_trapeze_p, 0).mean(axis=1)}, index=t)
        # Intervalo de confianza
        sigma_est_put = put.sem().Prima
        mean_est_put = put.iloc[-1].Prima
        i1_put = st.norm.interval(Trust_level, loc=mean_est_put, scale=sigma_est_put)
        
        end_p = time.time()
        total_time_p = end_p - start
    
        return put.iloc[-1].Prima, i1_put[0], i1_put[1], i1_put[1] - i1_put[0], total_time_p

$$\hat V_0^{(1)}= {e^{-rT} \over M} \sum_{j=1}^{M} \Bigg({1\over N} \sum_{i=0}^{N-1} S_{t_i}-K \Bigg)_+$$

In [4]:
# Función Riemann.
def Riemann_approach(K:'Strike price',
                     r:'Tasa libre de riesgo',
                     S0:'Precio inicial',
                     NbTraj:'Número trayectorias',
                     NbStep:'Cantidad de pasos a simular',
                     sigma:'Volatilidad',T:'Tiempo de cierre del contrato en años',
                     Option_Type: 'Tipo de opción a valuar. Call o Put.',
                     Trust_level: 'Confianza definida para el intervalo de precios',
                     flag=None):
    # Definimos tiempo de ejecución
    start = time.time()
    
    # Resolvemos la ecuación de black scholes para obtener los precios
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    # Almacenamos los precios en un dataframe
    prices = pd.DataFrame(St,index=t)
    # Obtenemos los precios promedios
    Average_t = prices.expanding().mean()
    # Definimos el dataframe de strikes
    strike = K
    
    if Option_Type in ['Call', 'call']:
        # Calculamos el call de la opción según la formula obtenida para Sumas de Riemann
        call = pd.DataFrame({'Prima': np.exp(-r*T) \
                     *np.fmax(Average_t - strike, 0).mean(axis=1)}, index=t)
        # Intervalo de confianza
        sigma_est_call = call.sem().Prima
        mean_est_call = call.iloc[-1].Prima
        i1_call = st.norm.interval(Trust_level, loc=mean_est_call, scale=sigma_est_call)
        
        end_c = time.time()
        total_time_c = end_c - start

        return call.iloc[-1].Prima, i1_call[0], i1_call[1], i1_call[1] - i1_call[0], total_time_c
    
    else:
        # Calculamos el put de la opción según la formula obtenida para Sumas de Riemann
        put = pd.DataFrame({'Prima': np.exp(-r*T) \
                     *np.fmax(strike - Average_t, 0).mean(axis=1)}, index=t)
        # Intervalo de confianza
        sigma_est_put = put.sem().Prima
        mean_est_put = put.iloc[-1].Prima
        i1_put = st.norm.interval(Trust_level, loc=mean_est_put, scale=sigma_est_put)
        
        end_p = time.time()
        total_time_p = end_p - start

        return put.iloc[-1].Prima, i1_put[0], i1_put[1], i1_put[1] - i1_put[0], total_time_p

In [5]:
# Parámetros
N_traj = [1000]*3+[5000]*3+[10000]*3+[50000]*3+[100000]*3+[500000]*3+[1000000]*3
N_step = [10,50,100]*len(set(N_traj))
index = [N_traj, N_step]

N_traj_sim = [1000, 5000, 10000, 50000, 100000, 500000, 1000000]
N_step_sim = [10, 50, 100]

# Valores de la opción.
S0 = 100
K = 100
r = 0.10
sigma = 0.20
T = 1

In [6]:
# Función para concatenar las trayectorias.
def valores_matriz(simulacion):
    
    # Trayectorias
    t1 = ([np.array(simulacion[i]) for i in range(len(simulacion))][0])
    t2 = ([np.array(simulacion[i]) for i in range(len(simulacion))][1])
    t3 = ([np.array(simulacion[i]) for i in range(len(simulacion))][2])
    t4 = ([np.array(simulacion[i]) for i in range(len(simulacion))][3])
    t5 = ([np.array(simulacion[i]) for i in range(len(simulacion))][4])
    t6 = ([np.array(simulacion[i]) for i in range(len(simulacion))][5])
    t7 = ([np.array(simulacion[i]) for i in range(len(simulacion))][6])
    m = np.concatenate([t1,t2,t3,t4,t5,t6,t7])
    
    return m

### Método Trapecio: `call asiática`

In [7]:
# Simulación para call con Método del Trapecio. 
trapecio_call_sim = [[Trapecio(K,r,S0,j,i,sigma,T,'Call',0.95) for i in N_step_sim] for j in N_traj_sim]                                         

# Data frame con la información.
trapecio_call = pd.DataFrame(valores_matriz(trapecio_call_sim),
                                 columns=['Aproximación', 'Linferior', 'Lsuperior', 'Longitud al 95%', 'Tiempo (mm:ss)'],
                                 index=pd.MultiIndex.from_arrays(index, names=('Tray. Montecarlo', 'Num. pasos en el tiempo')))
trapecio_call

Aproximación  Linferior  \
Tray. Montecarlo Num. pasos en el tiempo                            
1000             10                              7.185      5.774   
                 50                              7.095      6.670   
                 100                             7.284      7.001   
5000             10                              6.805      5.472   
                 50                              6.953      6.541   
...                                                ...        ...   
500000           50                              7.026      6.610   
                 100                             7.016      6.740   
1000000          10                              6.890      5.539   
                 50                              7.016      6.602   
                 100                             7.031      6.754   

                                          Lsuperior  Longitud al 95%  \
Tray. Montecarlo Num. pasos en el tiempo                               
1000             10                           8.595            2.821   
                 50                           7.521            0.850   
                 100                          7.568            0.567   
5000             10                           8.139            2.668   
                 50                           7.365            0.824   
...                                             ...              ...   
500000           50                           7.441            0.830   
                 100                          7.292            0.551   
1000000          10                           8.241            2.703   
                 50                           7.430            0.829   
                 100                          7.307            0.553   

                                          Tiempo (mm:ss)  
Tray. Montecarlo Num. pasos en el tiempo                  
1000             10                                0.077  
                 50                                0.065  
                 100                               0.091  
5000             10                                0.270  
                 50                                0.282  
...                                                  ...  
500000           50                               34.224  
                 100                              44.772  
1000000          10                               69.174  
                 50                               73.825  
                 100                              91.397  

[21 rows x 5 columns]

### Riemann: `call asiática`

In [9]:
# Simulación para call con Riemann.
riemann_call_sim = [[Riemann_approach(K,r,S0,j,i,sigma,T,'Call', 0.95) for i in N_step_sim] for j in N_traj_sim]

# Data frame con la información.
riemann_call = pd.DataFrame(valores_matriz(riemann_call_sim),
                                 columns=['Aproximación', 'Linferior', 'Lsuperior', 'Longitud al 95%', 'Tiempo(mm:ss)'],
                                 index=pd.MultiIndex.from_arrays(index, names=('Tray. Montecarlo', 'Num. pasos en el tiempo')))
riemann_call 

Aproximación  Linferior  \
Tray. Montecarlo Num. pasos en el tiempo                            
1000             10                              6.507      5.206   
                 50                              7.272      6.730   
                 100                             6.864      6.515   
5000             10                              6.542      5.243   
                 50                              7.007      6.485   
...                                                ...        ...   
500000           50                              6.908      6.393   
                 100                             6.981      6.622   
1000000          10                              6.412      5.140   
                 50                              6.920      6.403   
                 100                             6.996      6.635   

                                          Lsuperior  Longitud al 95%  \
Tray. Montecarlo Num. pasos en el tiempo                               
1000             10                           7.807            2.601   
                 50                           7.814            1.084   
                 100                          7.212            0.697   
5000             10                           7.841            2.598   
                 50                           7.528            1.043   
...                                             ...              ...   
500000           50                           7.424            1.031   
                 100                          7.341            0.719   
1000000          10                           7.683            2.543   
                 50                           7.436            1.033   
                 100                          7.357            0.722   

                                          Tiempo(mm:ss)  
Tray. Montecarlo Num. pasos en el tiempo                 
1000             10                               0.057  
                 50                               0.062  
                 100                              0.065  
5000             10                               0.271  
                 50                               0.284  
...                                                 ...  
500000           50                              38.982  
                 100                             35.533  
1000000          10                              54.764  
                 50                              55.593  
                 100                             66.026  

[21 rows x 5 columns]

### Método Trapecio: `put asiática`

In [10]:
# Simulación para put con Método del Trapecio.                                          
trapecio_put_sim = [[Trapecio(K,r,S0,j,i,sigma,T,'Put',0.95) for i in N_step_sim] for j in N_traj_sim]

# Data frame con la información.
trapecio_put = pd.DataFrame(valores_matriz(trapecio_put_sim),
                                 columns=['Aproximación', 'Linferior', 'Lsuperior', 'Longitud al 95%', 'Tiempo (mm:ss)'],
                                 index=pd.MultiIndex.from_arrays(index, names=('Tray. Montecarlo', 'Num. pasos en el tiempo')))
trapecio_put

Aproximación  Linferior  \
Tray. Montecarlo Num. pasos en el tiempo                            
1000             10                              2.223    -14.913   
                 50                              2.209     -5.272   
                 100                             2.388     -2.858   
5000             10                              2.195    -14.923   
                 50                              2.375     -5.086   
...                                                ...        ...   
500000           50                              2.334     -5.127   
                 100                             2.340     -2.918   
1000000          10                              2.226    -14.890   
                 50                              2.333     -5.130   
                 100                             2.341     -2.917   

                                          Lsuperior  Longitud al 95%  \
Tray. Montecarlo Num. pasos en el tiempo                               
1000             10                          19.359           34.272   
                 50                           9.690           14.962   
                 100                          7.633           10.491   
5000             10                          19.314           34.237   
                 50                           9.835           14.921   
...                                             ...              ...   
500000           50                           9.796           14.923   
                 100                          7.598           10.516   
1000000          10                          19.342           34.231   
                 50                           9.795           14.925   
                 100                          7.599           10.515   

                                          Tiempo (mm:ss)  
Tray. Montecarlo Num. pasos en el tiempo                  
1000             10                                0.100  
                 50                                0.115  
                 100                               0.074  
5000             10                                0.270  
                 50                                0.312  
...                                                  ...  
500000           50                               29.853  
                 100                              36.790  
1000000          10                               50.251  
                 50                               64.285  
                 100                              80.853  

[21 rows x 5 columns]

### Riemann: `put asiática`

In [11]:
# Simulación para call con Riemann.
riemann_put_sim = [[Riemann_approach(K,r,S0,j,i,sigma,T,'Put',0.95) for i in N_step_sim] for j in N_traj_sim]

# Data frame con la información.
riemann_put = pd.DataFrame(valores_matriz(riemann_put_sim),
                                 columns=['Aproximación', 'Linferior', 'Lsuperior', 'Longitud al 95%', 'Tiempo(mm:ss)'],
                                 index=pd.MultiIndex.from_arrays(index, names=('Tray. Montecarlo', 'Num. pasos en el tiempo')))
riemann_put

Aproximación  Linferior  \
Tray. Montecarlo Num. pasos en el tiempo                            
1000             10                              2.114      1.717   
                 50                              2.287      2.142   
                 100                             2.364      2.261   
5000             10                              2.292      1.850   
                 50                              2.251      2.113   
...                                                ...        ...   
500000           50                              2.330      2.183   
                 100                             2.336      2.236   
1000000          10                              2.222      1.796   
                 50                              2.326      2.178   
                 100                             2.347      2.247   

                                          Lsuperior  Longitud al 95%  \
Tray. Montecarlo Num. pasos en el tiempo                               
1000             10                           2.512            0.796   
                 50                           2.431            0.290   
                 100                          2.466            0.205   
5000             10                           2.734            0.884   
                 50                           2.389            0.276   
...                                             ...              ...   
500000           50                           2.478            0.295   
                 100                          2.435            0.199   
1000000          10                           2.648            0.851   
                 50                           2.473            0.295   
                 100                          2.447            0.200   

                                          Tiempo(mm:ss)  
Tray. Montecarlo Num. pasos en el tiempo                 
1000             10                               0.101  
                 50                               0.085  
                 100                              0.092  
5000             10                               0.319  
                 50                               0.466  
...                                                 ...  
500000           50                              26.672  
                 100                             30.671  
1000000          10                            1256.268  
                 50                              74.770  
                 100                             71.764  

[21 rows x 5 columns]

> Los reusltados obtenidos en las tablas anteriores varían un poco. Sin embargo, creo que se pueda considerar como una diferencia insignificativa.

> Respecto a los tiempos en ambos métodos para los dos tipos de opciones son muy similares, por lo que en términos de optimización, es irrelevante el método que se escoja usar.